<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/Supervised_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crimes Data in Israel (2019-2024)

#### by: Eden Shmuel, Oriana Feltzer

***

# Importing necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import glob

In [ ]:
!pip install nbimporter

In [ ]:
import nbimporter
from Data_Handling import Preprocessing

ModuleNotFoundError: No module named 'Data_Handling'

Define Paths to Crime Datasets (2019–2024)

In [ ]:
!git clone https://github.com/orifelszer/CrimeData.git

import os
import zipfile

zip_folder = 'CrimeData'

zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]

for zip_file in zip_files:
    zip_path = os.path.join(zip_folder, zip_file)
    extract_path = os.path.join(zip_folder, zip_file.replace('.zip', ''))

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted: {zip_file} -> {extract_path}")

fatal: destination path 'CrimeData' already exists and is not an empty directory.
Extracted: crimes2022.zip -> CrimeData/crimes2022
Extracted: crimes2020.zip -> CrimeData/crimes2020
Extracted: crimes2019.zip -> CrimeData/crimes2019
Extracted: crimes2024.zip -> CrimeData/crimes2024
Extracted: crimes2021.zip -> CrimeData/crimes2021
Extracted: crimes2023.zip -> CrimeData/crimes2023


In [ ]:
csv_files = glob.glob("CrimeData/**/*.csv", recursive=True)
dataframes = {f"Crimes_{file[-8:-4]}": pd.read_csv(file) for file in csv_files}

Loaded 6 datasets successfully.


Load Crime Data into Pandas DataFrames

Preview 2019 Crime Data

In [ ]:
combined_data = pd.concat(dataframes, axis=0, ignore_index=True)

,FictiveIDNumber,Year,Quarter,YeshuvKod,Yeshuv,PoliceDistrictKod,PoliceDistrict,PoliceMerhavKod,PoliceMerhav,PoliceStationKod,PoliceStation,municipalKod,municipalName,StatisticAreaKod,StatisticArea,StatisticGroupKod,StatisticGroup,StatisticTypeKod,StatisticType
0,FB00D9C786EA3D2B9730A55C86FAC35E,2024,Q2,7000.0,לוד,40000000,מחוז מרכז,41200000,מרחב שפלה,41213000,תחנת לוד שפלה,NaN,NaN,70000015.0,רמת אשכול שבזי העיר העתיקה (מזרח),200,עבירות סדר ציבורי,206,איומים
1,0D08AB3E39C02AA009F27EAD1C52CB06,2024,Q3,510.0,כפר מנדא,10000000,מחוז צפון,15002736,מרחב גליל צפון,12401000,תחנת משגב גליל צפון,NaN,NaN,NaN,NaN,100,עבירות בטחון,106,עבירות אחרות כלפי בטחון
2,58A5A580C153F7BA3FACA2CA0435805A,2024,Q1,3000.0,ירושלים,50000000,מחוז ירושלים,51200000,מרחב ציון ירושלים,51300000,תחנת מוריה ציון,NaN,NaN,30001221.0,גבעת מרדכי,800,עבירות מרמה,803,מרמה ועושק
3,C69077BBD4AAAF9D7906B9854EEB4DB6,2024,Q2,NaN,NaN,10000000,מחוז צפון,15002690,מרחב עמקים צפון,12101000,תחנת מגדל העמק עמקים צפון,9.0,מועצה אזורית עמק יזרעאל,NaN,NaN,700,עבירות כלפי הרכוש,715,גניבות אחרות
4,B02F812C0BCCEDB9D03FC1DA62642CB4,2024,Q2,7600.0,עכו,80000000,מחוז חוף,15002761,מרחב אשר חוף,81601000,תחנת עכו מחוז חוף,NaN,NaN,76000013.0,בן גוריון,700,עבירות כלפי הרכוש,719,גרימת נזק לרכוש בזדון
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341168,43302FEBB344EDA7F69A215321E05984,2019,Q2,4000.0,חיפה,80000000,מחוז חוף,15005216,מרחב כרמל חוף,81100000,תחנת חיפה מחוז חוף,NaN,NaN,NaN,NaN,800,עבירות מרמה,803,מרמה ועושק
2341169,C1E294CF1FF5BAC5F344AD985320B070,2019,Q3,NaN,NaN,70000000,מחוז שי,71500000,מרחב שומרון שי,71600800,תחנת מודיעין עילית שומרון,NaN,ישוב פלסטיני,NaN,NaN,100,עבירות בטחון,106,עבירות אחרות כלפי בטחון
2341170,4CEEA9121217A31B6F29B2C8C5065A08,2019,Q3,7100.0,אשקלון,30000000,מחוז דרומי,31200000,מרחב לכיש,31210000,תחנת אשקלון לכיש,NaN,NaN,71000113.0,נאות ברנע,700,עבירות כלפי הרכוש,719,גרימת נזק לרכוש בזדון
2341171,3C4D107C8ACC6CC8C6C59A58EA84C41B,2019,Q4,6400.0,הרצליה,20000000,מחוז תא,21700000,מרחב ירקון תא,21702000,תחנת גלילות ירקון,NaN,NaN,64000064.0,הרצליה פיתוח (מזרח),700,עבירות כלפי הרכוש,715,גניבות אחרות


In [ ]:
# הסרת שורות עם ערך -1 בעמודת StatisticGroupKod
combined_data = combined_data[combined_data['StatisticGroupKod'] != -1]

בדיקת חריגים לפני חלוקה לאימון ומבחן

In [ ]:
# שמירת עמודות החיזוי
statistic_group = combined_data['StatisticGroup']
statistic_group_kod = combined_data['StatisticGroupKod']

# הסרת עמודות החיזוי מהנתונים
features = combined_data.drop(['StatisticGroup', 'StatisticGroupKod'], axis=1)

# חלוקה לסט אימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(
    features,
    statistic_group,  # משתמשים רק ב-StatisticGroupKod כעמודת חיזוי
    test_size=0.2,
    random_state=42,
    stratify=statistic_group  # שומר על התפלגות הקטגוריות
)

# שמירת עמודת StatisticGroup (לשימוש אפשרי בעתיד)
y_train_text = statistic_group_kod.loc[X_train.index]
y_test_text = statistic_group_kod.loc[X_test.index]

In [ ]:
# ניקוי סט האימון ויצירת מיפויים
X_train_cleaned, train_mappings, scaler = Preprocessing(X_train, fit_scaler=True)

# ניקוי סט הבדיקה תוך שימוש במיפויים שנוצרו בסט האימון
X_test_cleaned, _, _ = Preprocessing(X_test, train_mappings=train_mappings, scaler=scaler)

In [ ]:
# הגדרת עמודות לשמירה על בסיס סט האימון בלבד
columns_to_keep = [col for col in X_train_cleaned.columns if X_train_cleaned[col].sum() >= 10]

# שמירה על עמודות אימון רלוונטיות בלבד
X_train_cleaned = X_train_cleaned[columns_to_keep]

# התאמת סט המבחן רק לפי מה שנקבע באימון
for col in columns_to_keep:
    if col not in X_test_cleaned.columns:
        X_test_cleaned[col] = 0

# הבטחה ששני הסטים באותו סדר עמודות
X_test_cleaned = X_test_cleaned[X_train_cleaned.columns]

In [ ]:
# נתונים לעבודה
X_train = X_train_cleaned
X_test = X_test_cleaned

In [ ]:
# התאמת אינדקסים
y_train = y_train.loc[X_train_cleaned.index]
y_test = y_test.loc[X_test_cleaned.index]

Models

In [ ]:
# Reducing memory usage
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
# ייבוא הספריות הדרושות
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from itertools import product

In [ ]:
# # דגימה של חלק מסט האימון בלבד
# X_train_sample = X_train.sample(frac=0.05, random_state=42)
# y_train_sample = y_train.loc[X_train_sample.index]

# # בחירת התכונות על סמך הדגימה בלבד
# selector = SelectKBest(score_func=chi2, k=20)
# selector.fit(X_train_sample, y_train_sample)

# # החלת הבחירה על כל סט האימון והבדיקה
# X_train_selected = selector.transform(X_train)
# X_test_selected = selector.transform(X_test)

In [ ]:
# # 📈 מודל בסיסי ללא שיפורים (Baseline)
# sgd_model = SGDClassifier(loss='log_loss', class_weight='balanced', max_iter=100, random_state=42)
# sgd_model.fit(X_train_selected, y_train)

# # 📊 חיזוי על סט הבדיקה
# y_pred_baseline = sgd_model.predict(X_test_selected)

# # ✅ הערכת ביצועי המודל
# accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
# print(f"Baseline Test Accuracy: {accuracy_baseline:.4f}")

In [ ]:
# # 🔁 קרוס-ולידציה על המודל הבסיסי
# cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# cv_scores = cross_val_score(sgd_model, X_train_selected, y_train, cv=cv, scoring='accuracy')

# # תוצאות
# print(f"Cross-Validation Scores: {cv_scores}")
# print(f"Average Accuracy: {np.mean(cv_scores):.4f}")

In [ ]:
# 📊 פונקציה למציאת הפרמטרים האופטימליים של SGDClassifier
def find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features, cv_folds=3):
    best_score = 0
    best_params = {}
    results = []

    # לולאה על כל הקומבינציות האפשריות
    for alpha, max_iter, tol, k in product(alphas, max_iters, tols, k_features):

        # 🧩 בחירת התכונות
        selector = SelectKBest(score_func=chi2, k=k)
        selector.fit(X_train, y_train)
        X_train_selected = selector.transform(X_train)
        X_test_selected = selector.transform(X_test)

        # 📈 בניית מודל SGD עם הפרמטרים שנבדקים
        model = SGDClassifier(
            loss='log_loss',
            alpha=alpha,
            max_iter=max_iter,
            tol=tol,
            random_state=42,
            class_weight='balanced'
        )

        # 🔁 קרוס-ולידציה
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv, scoring='f1_macro')

        # אימון המודל על כל הדאטה
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        # 📊 חישוב מדדים
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        # שמירת התוצאות בכל הרצה
        results.append({
            'alpha': alpha,
            'max_iter': max_iter,
            'tol': tol,
            'features': k,
            'cv_f1_score': np.mean(cv_scores),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

        # בדיקה אם התוצאה הנוכחית היא הטובה ביותר
        if f1 > best_score:
            best_score = f1
            best_params = {
                'alpha': alpha,
                'max_iter': max_iter,
                'tol': tol,
                'features': k,
                'f1': f1
            }

    # ✅ הדפסת התוצאה הטובה ביותר
    print(f"\n📊 Best Parameters Found:")
    print(f"Alpha: {best_params['alpha']}, Max Iter: {best_params['max_iter']}, Tol: {best_params['tol']}, Features: {best_params['features']}")
    print(f"Best F1-Score: {best_params['f1']:.4f}")

    # ✅ החזרת כל התוצאות והפרמטרים הטובים ביותר
    return results, best_params

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-99a2b43afd8a>", line 79, in <cell line: 79>
    results, best_params = find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features)
  File "<ipython-input-24-99a2b43afd8a>", line 31, in find_best_sgd_combination
    model.fit(X_train_selected, y_train)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py", line 932, in fit
    return self._fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py", line 719, in _fit
    self._partial_fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_g

TypeError: object of type 'NoneType' has no len()

In [ ]:
# דוגמה לפרמטרים אפשריים לבדיקה
alphas = [0.0001, 0.001, 0.01]
max_iters = [100, 300]
tols = [1e-3, 1e-4]
k_features = [10, 20, 30]

# קריאה לפונקציה עם כל הקומבינציות
results, best_params = find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features)

# הדפסת הקומבינציה הטובה ביותר
print("Best Parameters:", best_params)


*****